In [1]:
import tensorflow as tf
import numpy as np
import input_data

In [2]:
mnist_width = 28
n_visible = mnist_width * mnist_width
n_hidden = 500
corruption_level = 0.3

In [3]:
# Create node for input data
X  = tf.placeholder("float", [None, n_visible], name='X')

# Create node for corruption mask
mask = tf.placeholder("float", [None, n_visible], name='mask')

# Create nodes for hidden variables
W_init_mask = 4 * np.sqrt(6 / (n_visible + n_hidden))
W_init = tf.random_uniform(shape=[n_visible, n_hidden], 
                           minval=-W_init_mask, maxval=W_init_mask)

In [4]:
W = tf.Variable(W_init, name='W')
b = tf.Variable(tf.zeros([n_hidden]), name='b')

In [5]:
W_prime = tf.transpose(W)
b_prime = tf.Variable(tf.zeros([n_visible]), name='b_prime')

In [6]:
def model(X, mask, W, b, W_prime, b_prime):
    corrupted_X = mask * X
    Y = tf.nn.sigmoid(tf.matmul(corrupted_X, W) + b)
    Z = tf.nn.sigmoid(tf.matmul(Y, W_prime) + b_prime)
    return Z

In [7]:
Z = model(X, mask, W, b, W_prime, b_prime)

In [8]:
cost = tf.reduce_sum(tf.pow(X - Z, 2))
train_op = tf.train.GradientDescentOptimizer(0.02).minimize(cost)

In [9]:
# Load MNIST data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
trX,  trY, teX, teY = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels

('Extracting', 'MNIST_data/train-images-idx3-ubyte.gz')
('Succesfully downloaded', 'train-labels-idx1-ubyte.gz', 28881, 'bytes.')
('Extracting', 'MNIST_data/train-labels-idx1-ubyte.gz')
('Succesfully downloaded', 't10k-images-idx3-ubyte.gz', 1648877, 'bytes.')
('Extracting', 'MNIST_data/t10k-images-idx3-ubyte.gz')
('Succesfully downloaded', 't10k-labels-idx1-ubyte.gz', 4542, 'bytes.')
('Extracting', 'MNIST_data/t10k-labels-idx1-ubyte.gz')


In [ ]:
with tf.Session() as sess:
    tf.initialize_all_variables().run()
    
    for i in range(100):
        for start, end in zip(range(0, len(trX), 128), range(128, len(trX), 128)):
            input_ = trX[start:end]
            mask_np = np.random.binomial(1, 1-corruption_level, input_.shape)
            sess.run(train_op, feed_dict={X: input_, mask: mask_np})
        
        mask_np = np.random.binomial(1, 1-corruption_level, teX.shape)
        print(i, sess.run(cost, feed_dict={X: teX, mask: mask_np}))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
(0, 505583.59)
(1, 502836.91)
(2, 500341.0)
(3, 498558.44)
(4, 496327.16)
(5, 495479.44)
(6, 494374.69)
